In [1]:
import numpy as np
import pandas as pd

In [2]:
class Node:
    def __init__(self, feature_index, split_value, left_indices, right_indices):
        self.feature_index = feature_index
        self.split_value = split_value
        self.left_indices = left_indices
        self.right_indices = right_indices
        self.left = None
        self.right = None
        
        
    def __str__(self):
        return 'values = ' + str(self.split_value) +', index = '+ str(self.feature_index)

In [3]:
def gini_impurity(array, classes):
    psquared = 0
    n = len(array)
    for x in classes:
        p_temp = np.sum(array == x)/n
        psquared+=p_temp*p_temp
    return 1 - psquared

def entropy(array, classes):
    ent = 0
    n = len(array)
    for x in classes:
        p_temp = np.sum(array == x)/n
        ent+=-p_temp*np.log(p_temp)
    return ent

def weighted_average(array1, array2, classes, p_1, p_2):
    if len(array1) == 0:
        gini_1 = 0
    else:
        gini_1 = gini_impurity(array1, classes)
    
    if len(array2) == 0:
        gini_2 = 0
    else:
        gini_2 = gini_impurity(array2, classes)

    return p_1*gini_1 + p_2*gini_2

def index( target_groups, classes):
    N = len(target_groups[0])+len(target_groups[1])
    p1,p2 = len(target_groups[0])/N, len(target_groups[1])/N
    return weighted_average( target_groups[0], target_groups[1], classes,p1,p2)




In [4]:
#new way

def generic_split(feature_index, value, Xdataset):
    column = Xdataset[:,feature_index]
    
    _all = set(range(0,len(column)))
    left = set(np.where(column < value)[0])
    right = _all - left
            
    return left, right

def get_data_from_idx(left, right, X, Y):
    assert(isinstance(left,set))
    assert(isinstance(right,set))
    
    ly, ry, lx, rx = Y[list(left)], Y[list(right)], X[list(left)], X[list(right)]
    return ly, ry, lx, rx

def best_split(Xdataset, Ydataset):
    N = len(Xdataset)
    feature_space = len(Xdataset[0])
    g = np.inf
    v = None
    feature = None
    group = []
    
    for i in range(0,N):
        for j in range(0,feature_space):
            left,right = generic_split(j,Xdataset[i][j],Xdataset)
            ly, ry, lx, rx = get_data_from_idx(left, right, Xdataset, Ydataset)
            g_index = index([ly,ry],list(set(Ydataset)))
            if g_index < g:
                g = g_index
                feature = j
                value = Xdataset[i][feature]
                group = [left,right]#[[lx,ly],[rx,ry]]
    result = Node(feature, value, left, right)#Node(feature, value, group)
    return result

In [27]:
def terminate_with_mode(targets):
    from collections import Counter
    c = Counter(targets)
    return c

In [67]:
def main_split(X,y, root, max_depth, min_size, depth):
    left_Y, right_Y, left_X, right_X = get_data_from_idx(root.left_indices, root.right_indices, X, y)
    
    
    if len(left_Y) == 0 or len(right_Y) == 0:
        root.left = root.right = terminate_with_mode(np.concatenate((left_Y,right_Y)))
        return

    if depth >= max_depth:
        root.left = terminate_with_mode(left_Y)
        root.right = terminate_with_mode(right_Y)
        return 

    if len(left_Y) <  min_size:
        root.left = terminate_with_mode(left_Y)
    else:
        root.left = best_split(left_X, left_Y)
        main_split(X,y, root.left, max_depth, min_size, depth+1)

    if len(right_Y) <  min_size:
        root.right = terminate_with_mode(right_Y)
    else:
        root.right = best_split(right_X, right_Y)
        main_split(X,y,root.right, max_depth, min_size, depth+1)


In [68]:
def build_tree(X_train,y_train, max_depth, min_size):
    root = best_split( X_train,y_train)
    main_split(X_train, y_train,root, max_depth, min_size, 1)
    return root

def predict(model, datapoint):
    value = model.split_value
    index = model.feature_index
    while True:
        if datapoint[index] <= value:
            model = model.left
            if isinstance(model,Node):
                value = model.split_value
                index = model.feature_index
            else:
                result = model
                break
        else:
            model = model.right
            if isinstance(model,Node):
                value = model.split_value
                index = model.feature_index
            else:
                result = model
                break
    return result

In [69]:
df = pd.read_csv('banknotes.txt', header =None)

df = df.sample(frac = 1)
df.columns = ['X_0','X_1','X_2','X_3','Y']
X = df[['X_0','X_1','X_2','X_3']].values
y = df['Y'].values



In [72]:
X = X[:100]
y = y[:100]

root = build_tree(X,y,100,2)

In [73]:
[(predict(root, X[i]),y[i]) for i in range(0,len(X))]

[(Counter({1: 4, 0: 5}), 1),
 (Counter({1: 4, 0: 5}), 1),
 (Counter({1: 4, 0: 5}), 1),
 (Counter({1: 17, 0: 38}), 0),
 (Counter({1: 17, 0: 38}), 0),
 (Counter({1: 17, 0: 38}), 0),
 (Counter({1: 17, 0: 38}), 0),
 (Counter({1: 17, 0: 38}), 0),
 (Counter({1: 4, 0: 5}), 1),
 (Counter({1: 17, 0: 38}), 0),
 (Counter({1: 17, 0: 38}), 0),
 (Counter({1: 17, 0: 38}), 1),
 (Counter({1: 1}), 0),
 (Counter({1: 4, 0: 5}), 1),
 (Counter({1: 4, 0: 5}), 1),
 (Counter({1: 1}), 0),
 (Counter({1: 4, 0: 5}), 1),
 (Counter({1: 17, 0: 38}), 0),
 (Counter({1: 4, 0: 5}), 1),
 (Counter({1: 17, 0: 38}), 0),
 (Counter({1: 17, 0: 38}), 0),
 (Counter({1: 17, 0: 38}), 0),
 (Counter({1: 4, 0: 5}), 1),
 (Counter({1: 17, 0: 38}), 0),
 (Counter({1: 17, 0: 38}), 0),
 (Counter({1: 17, 0: 38}), 0),
 (Counter({1: 4, 0: 5}), 1),
 (Counter({1: 17, 0: 38}), 1),
 (Counter({1: 17, 0: 38}), 0),
 (Counter({1: 17, 0: 38}), 0),
 (Counter({1: 17, 0: 38}), 0),
 (Counter({1: 17, 0: 38}), 0),
 (Counter({1: 17, 0: 38}), 0),
 (Counter({1:

In [37]:
dataset = [[2.771244718,1.784783929,0],
	[1.728571309,1.169761413,0],
	[3.678319846,2.81281357,0],
	[3.961043357,2.61995032,0],
	[2.999208922,2.209014212,0],
	[7.497545867,3.162953546,1],
	[9.00220326,3.339047188,1],
	[7.444542326,0.476683375,1],
	[10.12493903,3.234550982,1],
	[6.642287351,3.319983761,1]]
dataset = np.array(dataset)

In [38]:
y = dataset[:,2]
X = dataset[:,0:2]

In [58]:
root = build_tree(X,y,2,1)

In [59]:
[(predict(root, X[i]),y[i]) for i in range(0,len(X))]

[(Counter({0.0: 5, 1.0: 2}), 0.0),
 (Counter({0.0: 5, 1.0: 2}), 0.0),
 (Counter({0.0: 5, 1.0: 2}), 0.0),
 (Counter({0.0: 5, 1.0: 2}), 0.0),
 (Counter({0.0: 5, 1.0: 2}), 0.0),
 (Counter({0.0: 1}), 1.0),
 (Counter({0.0: 1}), 1.0),
 (Counter({0.0: 1}), 1.0),
 (Counter({0.0: 1}), 1.0),
 (Counter({0.0: 5, 1.0: 2}), 1.0)]

In [233]:
root.left.left

AttributeError: 'ModeResult' object has no attribute 'left'